---

---

# Load version in files and extract data from their commits

---

---

## File packages

### pip installed packages

In [1]:
import os.path

import git as git
import pandas as pd
from configparser import ConfigParser

 ### Local packages


In [2]:
import src.versions as my_versions

---

## Constants loading


In [3]:
config: ConfigParser = ConfigParser()
config.read("config.ini")

data_directory: str = config["GENERAL"]["DataDirectory"]
hive_git_directory: str = config["GIT"]["HiveGitDirectory"]
hive_git_repo_Name: str = config["GIT"]["HiveGitRepoName"]

hive_git_path: str = os.path.join(data_directory, hive_git_directory, hive_git_repo_Name)

repo: git.Repo = git.Repo(hive_git_path)

---

## Extracting git tags of minor and major versions

Alpha, beta and patch are not selected

In [4]:
tags = my_versions.get_versions_tags(repo)
tags

[<git.TagReference "refs/tags/rel/release-2.1.0">,
 <git.TagReference "refs/tags/rel/release-2.2.0">,
 <git.TagReference "refs/tags/rel/release-2.3.0">,
 <git.TagReference "refs/tags/rel/release-3.0.0">,
 <git.TagReference "refs/tags/rel/release-3.1.0">,
 <git.TagReference "refs/tags/rel/release-4.0.0">,
 <git.TagReference "refs/tags/release-2.0.0">]

In [5]:
my_versions.order_versions(tags)

[<git.TagReference "refs/tags/release-2.0.0">,
 <git.TagReference "refs/tags/rel/release-2.1.0">,
 <git.TagReference "refs/tags/rel/release-2.2.0">,
 <git.TagReference "refs/tags/rel/release-2.3.0">,
 <git.TagReference "refs/tags/rel/release-3.0.0">,
 <git.TagReference "refs/tags/rel/release-3.1.0">,
 <git.TagReference "refs/tags/rel/release-4.0.0">]

---

---

# Data processing using understand from scitools


---

---

##